## An idea of the visualise map tool
UI isn't that great. woulf be nice to build out a cleaner one.

In [1]:
# trying to make a function out of it

import altair as alt
import json
import folium
from gpxplotter import create_folium_map, read_gpx_file, add_segment_to_map
import numpy as np
import base64
from altair_saver import save
import pandas as pd
from folium.features import DivIcon


line_options = {'weight': 8}


the_map = create_folium_map(tile = 'OpenStreetMap')


def visualise_gpx(the_map, filename, segment_name = 'Bike Ride', tile = 'stamenterrain'):

    for track in read_gpx_file(filename):
        #print(track)
        for i, segment in enumerate(track['segments']):
            add_segment_to_map(the_map, segment,
                            cmap='viridis', line_options=line_options)

        # Create a chart using Altair
        idx = len(segment['elevation'])

        data = {
            'x': segment['Distance / km'],
            'y': segment['elevation'],
        }
        # Convert the data to a Pandas DataFrame
        df = pd.DataFrame(data)

        # Specify the data type for the x encoding field
        line = alt.Chart(df).mark_line().encode(
            x=alt.X('x', title='Distance / km'),
            y=alt.Y('y', title='Elevation / m')
        )

        WIDTH = 400
        HEIGHT = 200

        line = line.properties(
            width=WIDTH,
            height=HEIGHT
        )

        # Save the chart as a PNG image
        #png_bytes = save(line, format='png')
        line.save('test.html')
        # Encode the PNG image as base64 string
        chart_html = open("test.html", "r").read()
        # Create the HTML content for the popup
        html = ''' <h1 style="font-family: Verdana"> {0}</h1><br>
                <p style="font-family: Verdana"> Distance: {1} </p>
                <p style="font-family: Verdana"> Total elevation Gain: {2} </p>
                <p style="font-family: Verdana"> Average Speed: {3} </p>
                <img src="data:image/png;base64,{4}" />

                <br>
                {4}
                '''.format(segment_name,str(np.round(segment['distance'][-1]/1000,2))+' km', str(np.round(segment['elevation-up'], 1)) +' m', str(np.round(np.mean(segment['Velocity / km/h']), 1)) + ' km/h', chart_html)

        iframe = folium.IFrame(html=html, width=500, height=500)
        popup = folium.Popup(iframe, width=500, height=500)

        folium.TileLayer(tile).add_to(the_map)

        marker = folium.Marker(
            location=segment['latlon'][int(np.round(len(segment['latlon'])/2))],
            popup=popup,
            icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 18pt">{}</div>'.format(str(track['name'])[2:-2])),
        )
        marker.add_to(the_map)

    # To store the map as a HTML page:
visualise_gpx(the_map, 'data/day_1.gpx')
visualise_gpx(the_map, 'data/day_2.gpx')
visualise_gpx(the_map, 'data/day_3.gpx')
visualise_gpx(the_map, 'data/day_4.gpx')
# To display the map in a Jupyter notebook:
the_map



### An idea of metrics
The way I deal with file paths is rough.


In [2]:
def get_trip_statistics(files):

    # define metrics to expose
    total_distance = 0
    total_elevation = 0
    average_speed = 0
    num_days = len(files)
    for file in files:
        for track in read_gpx_file(file):
            segment = track['segments'][0] # this assumes that there is only one segment
            total_distance += segment['distance'][-1]/1000
            total_elevation += segment['elevation-up']
            average_speed += np.mean(segment['Velocity / km/h'])

    metrics_dict = {
        'Total Distance (km)':np.round(total_distance,2),
        'Total Elevation (m)':np.round(total_elevation,1),
        'Average Speed (km/h)':np.round(average_speed/num_days,1)}
    return metrics_dict

paths = ['data/day_1.gpx', 'data/day_2.gpx', 'data/day_3.gpx', 'data/day_4.gpx']
get_trip_statistics(paths)

{'Total Distance (km)': 586.6,
 'Total Elevation (m)': 2639.9,
 'Average Speed (km/h)': 22.7}